In [19]:
!pip install transformers datasets sentencepiece evaluate nltk rouge-score bert-score

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for bert-score from https://files.pythonhosted.org/packages/c6/8c/bc5457de4c004b1a623b31f7bc8d0375fb699b7d67df11879098b4b7b7c8/bert_score-0.3.13-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import evaluate

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Process ForkProcess-2:
KeyboardInterrupt
Process ForkProcess-1:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 102, in get
    with self._rlock:
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
KeyboardInterrupt
  File "/usr/lib/python3.10/multiprocess

In [21]:
checkpoint = 'AlanRobotics/ruT5-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [22]:
raw_dataset = load_dataset('sberquad')
raw_dataset['train'][0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [23]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [24]:
max_length = 512
def preprocess_dataset(example):
    tokenized_answers = tokenizer(example['answers']['text'][0])
    tokenized_context = tokenizer(example['context'], example['question'], max_length=max_length, truncation=True)
    tokenized_context['labels'] = tokenized_answers['input_ids']
    return tokenized_context

In [25]:
tokenized_dataset = raw_dataset.map(preprocess_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'title', 'context', 'question', 'answers'])
tokenized_dataset.set_format('torch')

In [27]:
batch_size = 16
logging_steps = len(tokenized_dataset["train"]) // batch_size
model_name = checkpoint.split('/')[-1]


bleu = evaluate.load('bleu', lang="ru")
rouge = evaluate.load('rouge', lang="ru")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": bleu_result, "rouge": rouge_result}


args = Seq2SeqTrainingArguments(
    output_dir=f'{model_name}-finetuned',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    report_to=None
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [29]:
trainer = Seq2SeqTrainer(
    model,
    args,
    data_collator,
    tokenized_dataset['train'],
    tokenized_dataset['validation'],
    tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [31]:
import torch, gc
import numpy as np
gc.collect()
torch.cuda.empty_cache()

In [32]:
trainer.train()

ClearML Task: overwriting (reusing) task id=b5b6b69c03594604a612e18a817f641e
ClearML results page: https://clearml-fundres.dev.mts.ai/projects/d18e8ef960cc4cd5a7cfa692f767935a/experiments/b5b6b69c03594604a612e18a817f641e/output/log


Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Epoch,Training Loss,Validation Loss,Bleu,Rouge
1,0.535800,0.335021,"{'bleu': 0.6509320210008414, 'precisions': [0.7598773499547602, 0.7036613272311213, 0.645869273528848, 0.5912574510360489], 'brevity_penalty': 0.9683414622302386, 'length_ratio': 0.9688321807733515, 'translation_length': 19894, 'reference_length': 20534}","{'rouge1': 0.22199546052306168, 'rouge2': 0.07128813237331905, 'rougeL': 0.2216512534730191, 'rougeLsum': 0.22198487417206753}"
2,0.399400,0.294594,"{'bleu': 0.6653125874486363, 'precisions': [0.8104740844614637, 0.7649759348389485, 0.7184782608695652, 0.6761363636363636], 'brevity_penalty': 0.8980841513625691, 'length_ratio': 0.9029414629395149, 'translation_length': 18541, 'reference_length': 20534}","{'rouge1': 0.2241746785779748, 'rouge2': 0.07199484061080569, 'rougeL': 0.22417070232300568, 'rougeLsum': 0.22429999122314448}"
3,0.363100,0.289390,"{'bleu': 0.6680823425466481, 'precisions': [0.8024984187223276, 0.7530855338691159, 0.7009997917100604, 0.6536400882445635], 'brevity_penalty': 0.9209662784350549, 'length_ratio': 0.9239310411999611, 'translation_length': 18972, 'reference_length': 20534}","{'rouge1': 0.225142957276595, 'rouge2': 0.07246158825289087, 'rougeL': 0.2249459023237815, 'rougeLsum': 0.22516508125622503}"


Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6182aedba0>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f61c94e20e0>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f617cbbdff0>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', p

TrainOutput(global_step=8499, training_loss=0.6092630286036639, metrics={'train_runtime': 2141.8579, 'train_samples_per_second': 63.489, 'train_steps_per_second': 3.968, 'total_flos': 7.179705434243072e+16, 'train_loss': 0.6092630286036639, 'epoch': 3.0})

In [40]:
context = 'И́лон Рив Маск род. 28 июня 1971[1][2][…], Претория, ЮАР) — американский предприниматель, инженер[5] и миллиардер. Основатель, генеральный директор и главный инженер компании SpaceX; инвестор, генеральный директор и архитектор продукта компании Tesla; основатель The Boring Company; соучредитель Neuralink и OpenAI; владелец Twitter. 7 января 2021 года, с состоянием по оценочным данным в 185 млрд $, впервые стал богатейшим человеком планеты, сместив на второе место основателя Amazon Джеффа Безоса[6]. 1 ноября 2021 года стал первым человеком в истории, чьё состояние достигло отметки в 300 млрд $[4][⇨]. Маск родился и вырос в Претории, ЮАР. Некоторое время учился в Преторийском университете, а в 17 лет переехал в Канаду. Поступил в Университет Куинс в Кингстоне и через два года перевёлся в Пенсильванский университет, где получил степень бакалавра по экономике и физике. В 1995 году переехал в Калифорнию, чтобы учиться в Стэнфордском университете, но вместо этого решил заняться бизнесом и вместе со своим братом Кимбалом  (англ.)рус. стал соучредителем компании Zip2, занимавшейся разработкой программного обеспечения для интернета. В 1999 году компания была приобретена Compaq за 307 миллионов долларов. В том же году Маск стал соучредителем онлайн-банка X.com, который в 2000 году конгломеративным путем консолидировался с Confinity и образовал PayPal. В 2002 году компания была куплена eBay за 1,5 миллиарда долларов. '
question = 'Какую компанию возглаваляет?'
tokenized_sentence = tokenizer(context, question, return_tensors='pt').to('cuda')
res = model.generate(**tokenized_sentence)
print(tokenizer.decode(res[0], skip_special_tokens=True))

SpaceX


In [79]:
from tqdm import tqdm

correct_answers = 0
num_answers = len(raw_dataset["validation"].select(range(1000)))
for row in tqdm(raw_dataset["validation"].select(range(1000))):
    tokenized_sentence = tokenizer(row["context"], row["question"], return_tensors='pt').to('cuda')
    res = model.generate(**tokenized_sentence)
    decoded_res = tokenizer.decode(res[0], skip_special_tokens=True)
    if decoded_res in row["answers"]["text"][0]:
        correct_answers += 1

print("ACCURACY: ", correct_answers/num_answers)

100%|██████████| 1000/1000 [01:53<00:00,  8.81it/s]

ACCURACY:  0.675


# SBERQUAD EXTENDED

In [90]:
from datasets import load_from_disk

In [91]:
dataset = load_from_disk("squad_extended")

In [92]:
dataset

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 49378
})

In [93]:
checkpoint = 'AlanRobotics/ruT5-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [94]:
max_length = 512
def preprocess_dataset(example):
    tokenized_answers = tokenizer(example['answers'])
    tokenized_context = tokenizer(example['context'], example['question'], max_length=max_length, truncation=True)
    tokenized_context['labels'] = tokenized_answers['input_ids']
    return tokenized_context

In [101]:
tokenized_dataset = dataset.map(preprocess_dataset)
tokenized_dataset = tokenized_dataset.remove_columns(['context', 'question', 'answers'])
tokenized_dataset.set_format('torch')

Map: 100%|██████████| 49378/49378 [01:37<00:00, 504.10 examples/s]


In [108]:
batch_size = 16
logging_steps = len(tokenized_dataset) // batch_size
model_name = checkpoint.split('/')[-1]


bleu = evaluate.load('bleu', lang="ru")
rouge = evaluate.load('rouge', lang="ru")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": bleu_result, "rouge": rouge_result}


args = Seq2SeqTrainingArguments(
    output_dir=f'{model_name}-finetuned',
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    report_to=None
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [109]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
raw_dataset = load_dataset('sberquad')
raw_dataset['train'][0]

max_length = 512
def preprocess_sber_dataset(example):
    tokenized_answers = tokenizer(example['answers']['text'][0])
    tokenized_context = tokenizer(example['context'], example['question'], max_length=max_length, truncation=True)
    tokenized_context['labels'] = tokenized_answers['input_ids']
    return tokenized_context

tokenized_sber_dataset = raw_dataset.map(preprocess_sber_dataset)
tokenized_sber_dataset = tokenized_dataset.remove_columns(['id', 'title', 'context', 'question', 'answers'])


In [106]:
tokenized_sber_dataset.set_format('torch')

In [111]:
trainer = Seq2SeqTrainer(
    model,
    args,
    data_collator,
    tokenized_dataset,
    tokenized_sber_dataset['validation'],
    tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [112]:
trainer.train()

ClearML Task: created new task id=f72b601ba1404eb49071ca1011707dc6
ClearML results page: https://clearml-fundres.dev.mts.ai/projects/d18e8ef960cc4cd5a7cfa692f767935a/experiments/f72b601ba1404eb49071ca1011707dc6/output/log


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Epoch,Training Loss,Validation Loss,Bleu,Rouge
1,0.462800,0.304165,"{'bleu': 0.6448765220773229, 'precisions': [0.779669005970462, 0.7269881917769242, 0.6698383466339267, 0.6159000913798355], 'brevity_penalty': 0.9273572903779463, 'length_ratio': 0.9298724067400409, 'translation_length': 19094, 'reference_length': 20534}","{'rouge1': 0.22329539496905043, 'rouge2': 0.07147938770258072, 'rougeL': 0.2229523613278227, 'rougeLsum': 0.22310493264127085}"
2,0.356000,0.297599,"{'bleu': 0.6657958785064494, 'precisions': [0.7797564687975647, 0.7261823633637726, 0.6692867540029112, 0.6128755364806867], 'brevity_penalty': 0.9590556373729314, 'length_ratio': 0.9598714327456901, 'translation_length': 19710, 'reference_length': 20534}","{'rouge1': 0.2252952066246347, 'rouge2': 0.07214853917594188, 'rougeL': 0.22477020761869848, 'rougeLsum': 0.22505215803964795}"
3,0.317200,0.289335,"{'bleu': 0.6683801503152508, 'precisions': [0.7858821125753397, 0.7331178654930546, 0.6771723122238587, 0.6218572881848569], 'brevity_penalty': 0.9523427097419066, 'length_ratio': 0.9534430700301938, 'translation_length': 19578, 'reference_length': 20534}","{'rouge1': 0.22637049264829245, 'rouge2': 0.07227541785722884, 'rougeL': 0.22598331088203993, 'rougeLsum': 0.22613583905283668}"


Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f61ac2c62f0>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f61ac22a680>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', port=8081): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6182741810>: Failed to establish a new connection: [Errno 113] No route to host'))
Exception encountered while uploading HTTPSConnectionPool(host='clearml-fundres.dev.mts.ai', p

TrainOutput(global_step=9261, training_loss=0.5718129830153943, metrics={'train_runtime': 3293.3155, 'train_samples_per_second': 44.98, 'train_steps_per_second': 2.812, 'total_flos': 7.735947668802765e+16, 'train_loss': 0.5718129830153943, 'epoch': 3.0})

In [114]:
context = 'И́лон Рив Маск род. 28 июня 1971[1][2][…], Претория, ЮАР) — американский предприниматель, инженер[5] и миллиардер. Основатель, генеральный директор и главный инженер компании SpaceX; инвестор, генеральный директор и архитектор продукта компании Tesla; основатель The Boring Company; соучредитель Neuralink и OpenAI; владелец Twitter. 7 января 2021 года, с состоянием по оценочным данным в 185 млрд $, впервые стал богатейшим человеком планеты, сместив на второе место основателя Amazon Джеффа Безоса[6]. 1 ноября 2021 года стал первым человеком в истории, чьё состояние достигло отметки в 300 млрд $[4][⇨]. Маск родился и вырос в Претории, ЮАР. Некоторое время учился в Преторийском университете, а в 17 лет переехал в Канаду. Поступил в Университет Куинс в Кингстоне и через два года перевёлся в Пенсильванский университет, где получил степень бакалавра по экономике и физике. В 1995 году переехал в Калифорнию, чтобы учиться в Стэнфордском университете, но вместо этого решил заняться бизнесом и вместе со своим братом Кимбалом  (англ.)рус. стал соучредителем компании Zip2, занимавшейся разработкой программного обеспечения для интернета. В 1999 году компания была приобретена Compaq за 307 миллионов долларов. В том же году Маск стал соучредителем онлайн-банка X.com, который в 2000 году конгломеративным путем консолидировался с Confinity и образовал PayPal. В 2002 году компания была куплена eBay за 1,5 миллиарда долларов. '
question = 'Какую компанию возглаваляет?'
tokenized_sentence = tokenizer(context, question, return_tensors='pt').to('cuda')
res = model.generate(**tokenized_sentence)
print(tokenizer.decode(res[0], skip_special_tokens=True))

SpaceX
